In [1]:
import numpy as np
import pandas as pd 
import shap
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from os import chdir 
from pathlib import Path
from os import getcwd
import pickle
chdir("C:\\Users\\afeki\\OneDrive\\Desktop\\code\\BootStrapDesicionTree\\")

%load_ext autoreload
%autoreload 2


In [2]:
from gradient_boosting_trees.gradient_boosting_regressor import CartGradientBoostingRegressor, \
    CartGradientBoostingRegressorKfold
from gradient_boosting_trees.gradient_boosting_abstract import GradientBoostingMachine
from Tree.node import Leaf

from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier

from sklearn.model_selection import train_test_split

import xgboost as xgb
import pickle

from catboost import Pool, CatBoostRegressor

# Config

In [3]:
results = {}

# Create simulated data
$$ y = a_1 \cdot x_1 + a_2 \cdot I(x_2 \in LEFT\_GROUP) + \sigma $$

$$ x1 - N(0,1) $$

In [4]:
def compute_mse(model_name,model,X,y, transform_x = True, categorical_features = None):
    if transform_x:
        temp_x = X.copy()
        if model_name == 'xgboost':
            temp_x = xgb.DMatrix(temp_x)
        elif model_name == 'catboost':
            if categorical_features:
                temp_x = Pool(temp_x , cat_features = categorical_features)
            else:
                temp_x = Pool(temp_x)
        return np.mean(np.square(y - model.predict(temp_x)))
    return np.mean(np.square(y - model.predict(X)))
    
    
cat_features=[2]

def permutation_feature_importance(model_name,model,X,y, categorical_features = None):
    results = {}
    mse = compute_mse(model_name, model,X,y, categorical_features = categorical_features)
    for col in X.columns:
        temp_x = X.copy()
        temp_x[col] = np.random.permutation(temp_x[col])
        if model_name == 'xgboost':
            temp_x = xgb.DMatrix(temp_x)
        elif model_name == 'catboost':
            if categorical_features:
                temp_x = Pool(temp_x , cat_features = categorical_features)
            else:
                temp_x = Pool(temp_x)
        new_mse = compute_mse(model_name, model,temp_x,y, transform_x = False, categorical_features = categorical_features)
        results[col] = new_mse - mse
    results = pd.Series(results)
    return results/results.sum()

def get_x1_x2_shap_value(model, x, columns = None):
    if columns is None:
        columns = x.columns
    abs_shap_values = pd.DataFrame(shap.TreeExplainer(model, feature_perturbation = "tree_path_dependent").shap_values(x), columns = columns).apply(np.abs)
    shap_values = abs_shap_values.mean()/abs_shap_values.mean().sum()
    return shap_values['x1'] + shap_values['x2']
    

# OUR IMPLEMENTATION

In [5]:
from utils import all_experiments, create_x_y, n_experiments, get_fitted_model, create_one_hot_x_x_val, create_mean_imputing_x_x_val
from config import N_EXPERIMENTS,N_ROWS, SIGMA, A1,VAL_RATIO, CATEGORY_COLUMN_NAME, Y_COL_NAME, MAX_DEPTH, LEARNING_RATE, N_ESTIMATORS,MODELS_DIR, A2, N_EXPERIMENTS, CATEGORIES
EXP_NAME = F"detect_uninformative_feature_{MAX_DEPTH}_nestimators_{N_ESTIMATORS}_learning_rate_{LEARNING_RATE}"

In [6]:
MODELS_DIR = Path(r"C:/Users/afeki/OneDrive/Desktop/code/BootStrapDesicionTree/experiments/detect_uninformative_feature_simulated_data/results/models/")
model_type= 'ours'
for exp_number, category_size in tqdm(all_experiments(), total=n_experiments):
    np.random.seed(exp_number)
    for predictor_name, predictor in {'Ffold': CartGradientBoostingRegressorKfold,'CartVanilla': CartGradientBoostingRegressor}.items():
        exp_name = F"{EXP_NAME}__{predictor_name}_exp_{exp_number}_category_size_{category_size}"
        model_path = MODELS_DIR / F"{exp_name}.pkl"
        X, y = create_x_y(category_size)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_RATIO, random_state=42)
        X_train[Y_COL_NAME] = y_train
        model = get_fitted_model(model_path, predictor, X_train, Y_COL_NAME)
        fi_gain = pd.Series(model.compute_feature_importance(method='gain')).sort_index()
        fi_gain /= fi_gain.sum()
        fi_permutation_train = permutation_feature_importance(model_type,model,X_train,y_train)
        fi_permutation_test = permutation_feature_importance(model_type,model, X_test, y_test)
        temp_results = np.array([fi_gain['x1'] + fi_gain['x2'],fi_permutation_train['x1'] + fi_permutation_train['x2'], fi_permutation_test['x1'] + fi_permutation_test['x2']])
        key = (F"{model_type}_{predictor_name}", category_size)
        results.setdefault(key,np.zeros(len(temp_results)))
        results[key] += temp_results

  0%|          | 0/345 [00:00<?, ?it/s]
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 345/345 [1:03:42<00:00, 11.08s/it]


## SKLEARN

In [7]:
pd.options.mode.chained_assignment = None 
model_type= 'sklearn'
for exp_number, category_size in tqdm(all_experiments(), total=n_experiments):
    np.random.seed(exp_number)
    for predictor_name, _ in {'one_hot': GradientBoostingRegressor,
                                      'mean_imputing': GradientBoostingRegressor}.items():
        X, y = create_x_y(category_size)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_RATIO, random_state=42)
        if predictor_name == 'one_hot':
            X_train, X_test = create_one_hot_x_x_val(X_train, X_test)
        elif predictor_name == 'mean_imputing':
            X_train, X_test = create_mean_imputing_x_x_val(X_train, y_train, X_test)
        
        model = GradientBoostingRegressor(max_depth=MAX_DEPTH, n_estimators=N_ESTIMATORS, learning_rate=LEARNING_RATE)
        model.fit(X_train, y_train)
        fi_gain = pd.Series(model.feature_importances_, index = X_train.columns)
        fi_gain /= fi_gain.sum()
        fi_permutation_train = permutation_feature_importance(model_type,model,X_train,y_train)
        fi_permutation_test = permutation_feature_importance(model_type,model, X_test, y_test)
        temp_results = np.array([fi_gain['x1'] + fi_gain['x2'],fi_permutation_train['x1'] + fi_permutation_train['x2'], 
                                 fi_permutation_test['x1'] + fi_permutation_test['x2'],get_x1_x2_shap_value(model, X_train), get_x1_x2_shap_value(model, X_test) ])
        key = (F"{model_type}_{predictor_name}", category_size)
        results.setdefault(key,np.zeros(len(temp_results)))
        results[key] += temp_results        

  0%|          | 0/345 [00:00<?, ?it/s]Setting feature_perturbation = "tree_path_dependent" because no background data was given.
The sklearn.ensemble.gradient_boosting module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
  0%|          | 1/345 [00:00<03:48,  1.50it/s]Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
  1%|          | 2/345 [00:01<03:47,  1.51it/s]Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
  1%|          | 3/345 [00:02<04

## XGBOOST

In [8]:
model_type= 'xgboost'
for exp_number, category_size in tqdm(all_experiments(), total=n_experiments):
    np.random.seed(exp_number)
    for predictor_name, _ in {'one_hot': GradientBoostingRegressor,
                                      'mean_imputing': GradientBoostingRegressor}.items():
        X, y = create_x_y(category_size)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_RATIO, random_state=42)
        if predictor_name == 'one_hot':
            X_train, X_test = create_one_hot_x_x_val(X_train, X_test)
        elif predictor_name == 'mean_imputing':
            X_train, X_test = create_mean_imputing_x_x_val(X_train,y_train, X_test)
        
        dtrain = xgb.DMatrix(X_train, label=y_train)
        param = {'max_depth': MAX_DEPTH - 1, 'eta': LEARNING_RATE, 'objective': 'reg:squarederror'}
        bst = xgb.train(param, dtrain, N_ESTIMATORS)
        fi_gain = pd.Series(bst.get_score(importance_type='gain'))
        fi_gain /= fi_gain.sum()
        fi_permutation_train = permutation_feature_importance(model_type,bst,X_train,y_train)
        fi_permutation_test = permutation_feature_importance(model_type,bst, X_test, y_test)
        temp_results = np.array([fi_gain['x1'] + fi_gain['x2'],fi_permutation_train['x1'] + fi_permutation_train['x2'], 
                                 fi_permutation_test['x1'] + fi_permutation_test['x2'],get_x1_x2_shap_value(bst, X_train), get_x1_x2_shap_value(bst, X_test) ])
        key = (F"{model_type}_{predictor_name}", category_size)
        results.setdefault(key,np.zeros(len(temp_results)))
        results[key] += temp_results      

100%|██████████| 345/345 [16:24<00:00,  2.85s/it]


## Catboost

In [9]:
# !pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [10]:
model_type= 'catboost'
for exp_number, category_size in tqdm(all_experiments(), total=n_experiments):
    np.random.seed(exp_number)
    for predictor_name in ["vanilla","mean_imputing"]:
        X, y = create_x_y(category_size)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_RATIO, random_state=42)
        if predictor_name == 'mean_imputing':
            X_train, X_test = create_mean_imputing_x_x_val(X_train,y_train, X_test)
            train_pool = Pool(X_train, y_train)
            val_pool = Pool(X_test,y_test)
        else:
            train_pool = Pool(X_train, y_train, cat_features=[2])
            val_pool = Pool(X_test,y_test, cat_features=[2])
    
        model = CatBoostRegressor(iterations=N_ESTIMATORS, 
                                  depth=MAX_DEPTH, 
                                  learning_rate=LEARNING_RATE, 
                                  loss_function='RMSE',logging_level = 'Silent')
        model.fit(train_pool)
        fi_gain = pd.Series(model.feature_importances_, index = model.feature_names_)
        fi_gain /= fi_gain.sum()
        cat_feature = [2] if predictor_name == 'vanilla' else None
        fi_permutation_train = permutation_feature_importance(model_type,model,X_train,y_train,  categorical_features = cat_feature)
        fi_permutation_test = permutation_feature_importance(model_type,model, X_test, y_test, categorical_features = cat_feature)
        temp_results = np.array([fi_gain['x1'] + fi_gain['x2'],fi_permutation_train['x1'] + fi_permutation_train['x2'], 
                         fi_permutation_test['x1'] + fi_permutation_test['x2'],get_x1_x2_shap_value(model, train_pool, columns = X_train.columns),get_x1_x2_shap_value(model, val_pool, columns = X_test.columns) ])
        key = (F"{model_type}_{predictor_name}", category_size)
        results.setdefault(key,np.zeros(len(temp_results)))
        results[key] += temp_results 

100%|██████████| 345/345 [02:51<00:00,  2.01it/s]


In [11]:
for key in results.keys():
    results[key] = results[key]/N_EXPERIMENTS

chdir("C:\\Users\\afeki\\OneDrive\\Desktop\\code\\BootStrapDesicionTree\\experiments\\detect_uninformative_feature_simulated_data\\")
with open('results/experiments_results/detect_uninformative_feature_results.pkl', 'wb') as output:
    pickle.dump(results, output, pickle.HIGHEST_PROTOCOL)